Automatic1111 WebUI

In [ ]:
# Main base
!apt update
!yes | apt install libgoogle-perftools-dev
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
%cd /kaggle/working/stable-diffusion-webui
!chmod +x webui.sh
#temporary folder for mpdels
!mkdir -p /kaggle/temp/models
#copy custom models to temp
import os
if os.path.exists('/kaggle/input/models'):
    !cp -r /kaggle/input/models/* /kaggle/temp/models/
else:
    print("Warning: No models found in /kaggle/input/models. Skipping copy.")
#temporary folder for control net
!mkdir -p /kaggle/temp/cnmodels
#ngrok install
!pip install -q pyngrok

In [ ]:
# ControlNet main
%cd /kaggle/working/stable-diffusion-webui/extensions
!git clone https://github.com/Mikubill/sd-webui-controlnet
%cd /kaggle/working/stable-diffusion-webui

#t2i-adapter_xl_openpose.safetensors
!wget -O /kaggle/temp/cnmodels/t2i-adapter_xl_openpose.safetensors "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors"
#t2i-adapter_xl_canny.safetensors
!wget -O /kaggle/temp/cnmodels/t2i-adapter_xl_canny.safetensors "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors"
#t2i-adapter_xl_sketch.safetensors
!wget -O /kaggle/temp/cnmodels/t2i-adapter_xl_sketch.safetensors "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors"
#t2i-adapter_diffusers_xl_depth_midas.safetensors
!wget -O /kaggle/temp/cnmodels/t2i-adapter_diffusers_xl_depth_midas.safetensors "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_midas.safetensors"
#t2i-adapter_diffusers_xl_depth_zoe.safetensors
!wget -O /kaggle/temp/cnmodels/t2i-adapter_diffusers_xl_depth_zoe.safetensors "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_zoe.safetensors"
#t2i-adapter_diffusers_xl_lineart.safetensors
!wget -O /kaggle/temp/cnmodels/t2i-adapter_diffusers_xl_lineart.safetensors "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors"

In [ ]:
#Copy LoRA from Kaggle Input to WebUI
!mkdir -p /kaggle/working/stable-diffusion-webui/models/Lora
!cp -r /kaggle/input/lorabs/* /kaggle/working/stable-diffusion-webui/models/Lora
import os
if os.path.exists('/kaggle/input/lorabs'):
    !cp -r /kaggle/input/lorabs/* /kaggle/working/stable-diffusion-webui/models/Lora
else:
    print("Warning: No LoRA found in /kaggle/input/lorabs/. Skipping copy.")

In [ ]:

#WebUI
from pyngrok import ngrok
from kaggle_secrets import UserSecretsClient
import os

secrets = UserSecretsClient()

NGROK_TOKEN = secrets.get_secret("NGROK_AUTHTOKEN")
WEBUI_PASSWORD = secrets.get_secret("PASSWORD")

ngrok.set_auth_token(NGROK_TOKEN)

#Starting WebUI
!./webui.sh \
  -f \
  --xformers \
  --listen \
  --port 7860 \
  --gradio-auth user:{WEBUI_PASSWORD} \
  --ckpt-dir "/kaggle/temp/models" \
  --enable-insecure-extension-access \
  --no-half-vae

In [ ]:
#Starting Public URL
public_url = ngrok.connect(7860)
print("Ngrok URL:", public_url)

In [ ]:
#zip archive for results
!rm -r /kaggle/working/generated_images.zip 
%cd /kaggle/working/stable-diffusion-webui/outputs/ 
!zip -r /kaggle/working/generated_images.zip * 
%cd -